In [17]:
# Final full script to merge ERA5, CHLA, TURB, LSWT, and export one CSV per lake

import os
import glob
import pandas as pd

def load_weekly_mean_files(folder, variable_name, required=True, lake_ids_filter=None):
    files = sorted(glob.glob(os.path.join(folder, f"*_{variable_name.upper()}_weekly.csv")))
    combined = []

    for file in files:
        try:
            df = pd.read_csv(file)
            df = df[["Lake_ID", "Year", "Week", "mean"]].copy()
            if lake_ids_filter is not None:
                df = df[df["Lake_ID"].isin(lake_ids_filter)]
            df.rename(columns={"mean": f"{variable_name.lower()}_mean"}, inplace=True)
            combined.append(df)
        except Exception as e:
            if required:
                print(f"[Required] Failed to process {file}: {e}")
            else:
                print(f"[Optional] Skipped {file} due to error: {e}")

    if combined:
        return pd.concat(combined, ignore_index=True)
    else:
        return pd.DataFrame()

def load_era5_data(era5_folder):
    files = sorted(glob.glob(os.path.join(era5_folder, "ERA5Land_Weekly_Catchments_*.csv")))
    dfs = []

    for file in files:
        df = pd.read_csv(file)
        df.rename(columns={"year": "Year", "week": "Week"}, inplace=True)
        dfs.append(df)

    return pd.concat(dfs, ignore_index=True)

def merge_all_sources_and_save(era5_folder, chla_folder, turb_folder, lswt_folder, output_folder):
    # Load ERA5 and get full list of Lake_IDs
    era5 = load_era5_data(era5_folder)
    valid_lakes = era5["Lake_ID"].unique()

    # Load CHLA, TURB (required), LSWT (optional)
    chla = load_weekly_mean_files(chla_folder, "CHLA", required=True, lake_ids_filter=valid_lakes)
    turb = load_weekly_mean_files(turb_folder, "TURB", required=True, lake_ids_filter=valid_lakes)
    lswt = load_weekly_mean_files(lswt_folder, "LSWT", required=False, lake_ids_filter=valid_lakes)


    # Find only lakes that are present in both CHLA and TURB files
    chla_ids = set(chla["Lake_ID"].unique())
    turb_ids = set(turb["Lake_ID"].unique())
    era5_ids = set(era5["Lake_ID"].unique())

    # Eligible = in CHLA ∩ TURB ∩ ERA5
    eligible_lake_ids = chla_ids & turb_ids & era5_ids

    # Excluded = (CHLA ∪ TURB) - eligible or not in ERA5
    excluded_lake_ids = (chla_ids | turb_ids) - eligible_lake_ids
    excluded_due_to_era5 = (chla_ids & turb_ids) - era5_ids

    # Print both groups
    missing_from_output = era5_ids - eligible_lake_ids
    print("Lakes in ERA5 but excluded due to missing CHLA or TURB:", sorted(missing_from_output))
    print(f"Count: {len(missing_from_output)}")


    # Filter ERA5 to only eligible lakes
    merged = era5[era5["Lake_ID"].isin(eligible_lake_ids)].copy()

    # Merge all variables
    merged = merged.merge(chla, on=["Lake_ID", "Year", "Week"], how="left")
    merged = merged.merge(turb, on=["Lake_ID", "Year", "Week"], how="left")
    merged = merged.merge(lswt, on=["Lake_ID", "Year", "Week"], how="left")

    # Create output folder
    os.makedirs(output_folder, exist_ok=True)

    # Save one CSV per eligible lake
    for lake_id in sorted(eligible_lake_ids):
        lake_df = merged[merged["Lake_ID"] == lake_id]
        output_path = os.path.join(output_folder, f"Lake_{lake_id}.csv")
        lake_df.to_csv(output_path, index=False)
        print(f"Saved: {output_path}")


# Example use:
# merge_all_sources_and_save("ERA5", "CHLA", "TURB", "LSWT", "Merged_Lake_CSVs")



In [18]:
result_df = merge_all_sources_and_save("Datasets/GEE/ECMWF_raw", "Datasets/CNR/weekly/CHLA", "Datasets/CNR/weekly/turbidity", "Datasets/CNR/weekly/LSWT", "Datasets/Merged_Lake_CSVs")


Lakes in ERA5 but excluded due to missing CHLA or TURB: [np.int64(2), np.int64(168), np.int64(337), np.int64(723), np.int64(300009430)]
Count: 5
Saved: Datasets/Merged_Lake_CSVs\Lake_5.csv
Saved: Datasets/Merged_Lake_CSVs\Lake_6.csv
Saved: Datasets/Merged_Lake_CSVs\Lake_12.csv
Saved: Datasets/Merged_Lake_CSVs\Lake_13.csv
Saved: Datasets/Merged_Lake_CSVs\Lake_15.csv
Saved: Datasets/Merged_Lake_CSVs\Lake_21.csv
Saved: Datasets/Merged_Lake_CSVs\Lake_26.csv
Saved: Datasets/Merged_Lake_CSVs\Lake_27.csv
Saved: Datasets/Merged_Lake_CSVs\Lake_28.csv
Saved: Datasets/Merged_Lake_CSVs\Lake_31.csv
Saved: Datasets/Merged_Lake_CSVs\Lake_37.csv
Saved: Datasets/Merged_Lake_CSVs\Lake_38.csv
Saved: Datasets/Merged_Lake_CSVs\Lake_44.csv
Saved: Datasets/Merged_Lake_CSVs\Lake_54.csv
Saved: Datasets/Merged_Lake_CSVs\Lake_57.csv
Saved: Datasets/Merged_Lake_CSVs\Lake_63.csv
Saved: Datasets/Merged_Lake_CSVs\Lake_68.csv
Saved: Datasets/Merged_Lake_CSVs\Lake_72.csv
Saved: Datasets/Merged_Lake_CSVs\Lake_76.csv
Sa